In [3]:
import os
from typing import Generic, Optional, SupportsFloat, Tuple, TypeVar, Union,Any
import torch
# import couple of libs some will be useful
import gym
import numpy as np
from collections import deque
import random
import re
import os
import sys
import time
import json
import torch as th
import itertools
from datasets import Dataset
from _code.const import PATH_MODEL_SB,PATH_DATA_INTERACTIONS,PATH_LOSS
from citylearn.agents.rbc import BasicRBC as BRBC
from agents.rbc import RBCAgent1 
# import stable_baselines3
from stable_baselines3 import PPO, A2C, DDPG, TD3,SAC
from stable_baselines3.common.utils import set_random_seed
import citylearn
from citylearn.citylearn import CityLearnEnv
from pathlib import Path
from rewards.combined_reward import CombinedReward
from stable_baselines3.common.utils import set_random_seed
ObsType = TypeVar("ObsType")
ActType = TypeVar("ActType")

import functools
from citylearn.wrappers import *
from utils.wrappers_custom import *
from datasets import load_from_disk,concatenate_datasets,Dataset

In [4]:
dataset= load_from_disk("data_interactions/winner_dataset_phase_1.pkl")

In [5]:
#best_action = np.load("single_agent_dp.npy",allow_pickle=True)
actions = np.array(dataset["actions"])

In [6]:
actions = actions.reshape(5,-1).T

In [8]:
actions.shape

(8759, 5)

In [20]:
#best_action[:,0].reshape(best_action.shape[0],1).tolist()

In [21]:
def action_space_to_dict(aspace):
    """ Only for box space """
    return {"high": aspace.high,
            "low": aspace.low,
            "shape": aspace.shape,
            "dtype": str(aspace.dtype)
            }


def env_reset(env):
    observations = env.reset()
    action_space = env.action_space
    observation_space = env.observation_space
    #building_info = env.buildings()
    #building_info = list(building_info.values())
    action_space_dicts = [action_space_to_dict(asp) for asp in action_space]
    observation_space_dicts = [action_space_to_dict(osp) for osp in observation_space]
    obs_dict = {"action_space": action_space_dicts,
                "observation_space": observation_space_dicts,
                #"building_info": building_info,
                "observation": observations}
    return obs_dict

In [22]:
#env= CityLearnEnv("citylearn_challenge_2022_phase_3",reward_function = RewardFunctionV1,buildings=[0,1,2,3,4,5,6])
env= CityLearnEnv("citylearn_challenge_2022_phase_1",reward_function = CombinedReward)
env = NormalizedObservationWrapperCustom(env)
env.central_agent = False
env.central_agent = False
agent= Agent()

In [23]:
obs_dict = env_reset(env)


In [24]:
#len(obs_dict["observation"][0])

In [25]:
#actions=agent.register_reset(obs_dict)

In [26]:
#env.current_obs

In [27]:
#counter = 0 

In [28]:
#actions = best_action[:,counter].reshape(best_action.shape[0],1).tolist()

In [29]:
#best_action[:,1]

In [30]:
counter=0
while True:
    #counter=0
    #actions = best_action[:,counter].reshape(best_action.shape[0],1).tolist()
    #actions = [np.array(act) for act in actions]
    observations,reward,done,_,_ = env.step(actions[counter].reshape(-1,1))
    counter = counter + 1
    
    if done :
        break
    #else:
        #actions= agent.compute_action(observations)

In [31]:
path_best = "data_interactions/best_dataset_multi.pkl"

In [32]:
data = env.dataset

In [33]:
env.evaluate()

/home/ml-stud15/anaconda3/envs/stable3/lib/python3.9/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.evaluate to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.evaluate` for environment variables or `env.get_wrapper_attr('evaluate')` that will search the reminding wrappers.
  logger.warn(


,cost_function,value,name,level
0,all_time_peak_average,0.834516,District,district
1,annual_normalized_unserved_energy_total,0.000000,District,district
2,carbon_emissions_total,0.830535,District,district
3,cost_total,0.730419,District,district
4,daily_one_minus_load_factor_average,0.908881,District,district
...,...,...,...,...
96,discomfort_hot_delta_maximum,0.000000,Building_5,building
97,discomfort_hot_delta_average,0.000000,Building_5,building
98,one_minus_thermal_resilience_proportion,NaN,Building_5,building
99,power_outage_normalized_unserved_energy_total,NaN,Building_5,building


In [34]:
Dataset.from_dict({k: [s[k] for s in data] for k in data[0].keys()}).save_to_disk(path_best)

Saving the dataset (0/1 shards):   0%|          | 0/8759 [00:00<?, ? examples/s]

In [ ]:
env.df_ts_eval.to_csv("data_result/RBC/train_TS.csv")

In [13]:
kpis_rbc = env.evaluate()
kpis_rbc = kpis_rbc.pivot(index='cost_function', columns='name', values='value').round(3)
kpis_rbc = kpis_rbc.dropna(how='all')
display(kpis_rbc)

/home/ml-stud15/anaconda3/envs/stable3/lib/python3.9/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.evaluate to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.evaluate` for environment variables or `env.get_wrapper_attr('evaluate')` that will search the reminding wrappers.
  logger.warn(


name,Building_1,Building_2,Building_3,Building_4,Building_5,District
cost_function,,,,,,
all_time_peak_average,NaN,NaN,NaN,NaN,NaN,0.569
annual_normalized_unserved_energy_total,0.000,0.000,0.000,0.000,0.000,0.000
carbon_emissions_total,0.752,0.787,0.675,0.897,0.758,0.774
cost_total,0.706,0.745,0.638,0.870,0.692,0.730
daily_one_minus_load_factor_average,NaN,NaN,NaN,NaN,NaN,0.807
daily_peak_average,NaN,NaN,NaN,NaN,NaN,0.502
discomfort_cold_delta_average,0.000,0.000,0.000,0.000,0.000,0.000
discomfort_cold_delta_maximum,0.000,0.000,0.000,0.000,0.000,0.000
discomfort_cold_delta_minimum,0.000,0.000,0.000,0.000,0.000,0.000


In [ ]:
env

In [ ]:
from stable_baselines3.sac import SAC as Agent
from citylearn.citylearn import CityLearnEnv
from citylearn.wrappers import NormalizedObservationWrapper, StableBaselines3Wrapper


In [ ]:
env = CityLearnEnv('citylearn_challenge_2023_phase_2_local_evaluation', central_agent=True)
env = NormalizedObservationWrapper(env)
env = StableBaselines3Wrapper(env)
model = Agent('MlpPolicy', env)

In [ ]:
episodes =50
model.learn(total_timesteps=env.unwrapped.time_steps*episodes)


In [ ]:
# test
observations= env.reset()
steps = 0
while True:
    actions, _ = model.predict(observations, deterministic=True)
    observations, _, done,_ = env.step(actions)
    steps +=1
    if done == True:
        break



In [ ]:
steps

In [ ]:
kpis = env.unwrapped.evaluate()
kpis = kpis.pivot(index='cost_function', columns='name', values='value').round(3)
kpis = kpis.dropna(how='all')
display(kpis)

## Data Interactions

In [5]:
from datasets import load_from_disk,concatenate_datasets,Dataset

In [6]:
dataset_best= load_from_disk("data_interactions/best_dataset_multi.pkl")
dataset_rbc = load_from_disk("data_interactions/RBCAgent1/model_RBCAgent1_timesteps_8760_rf_CombinedReward_phase_1.pkl")

In [13]:
dataset_best

Dataset({
    features: ['observations', 'next_observations', 'actions', 'rewards', 'dones', 'info'],
    num_rows: 8759
})

In [19]:
np.array(dataset_dones["rewards"]).shape

NameError: name 'dataset_dones' is not defined

In [36]:
combined_dataset = concatenate_datasets([dataset_best,dataset_rbc])

In [37]:
combined_path = ("data_interactions/combined_dataset_multi_agent.pkl")

In [38]:
Dataset.from_dict({k: [s[k] for s in combined_dataset] for k in combined_dataset[0].keys()}).save_to_disk(combined_path)


Saving the dataset (0/1 shards):   0%|          | 0/17518 [00:00<?, ? examples/s]